In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, Normalizer, StandardScaler
from sklearn import model_selection, tree, neighbors, metrics, ensemble
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
df_train= pd.read_csv('LFTS_train/train.csv')
df_test= pd.read_csv('LFTS_test.csv')

In [3]:
df_train.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [5]:
pd.isnull(df_train).sum()

UniqueID                                  0
disbursed_amount                          0
asset_cost                                0
ltv                                       0
branch_id                                 0
supplier_id                               0
manufacturer_id                           0
Current_pincode_ID                        0
Date.of.Birth                             0
Employment.Type                        7661
DisbursalDate                             0
State_ID                                  0
Employee_code_ID                          0
MobileNo_Avl_Flag                         0
Aadhar_flag                               0
PAN_flag                                  0
VoterID_flag                              0
Driving_flag                              0
Passport_flag                             0
PERFORM_CNS.SCORE                         0
PERFORM_CNS.SCORE.DESCRIPTION             0
PRI.NO.OF.ACCTS                           0
PRI.ACTIVE.ACCTS                

In [6]:
df_train['Employment.Type'].fillna(df_train['Employment.Type'].dropna().max(), inplace=True)
df_test['Employment.Type'].fillna(df_test['Employment.Type'].dropna().max(), inplace=True)

In [9]:
le=LabelEncoder()
col=['PERFORM_CNS.SCORE.DESCRIPTION', 'Employment.Type', 'Employee_code_ID']
for c in col:
    df_train[c]=le.fit_transform(df_train[c])
    df_test[c]=le.fit_transform(df_test[c])

In [20]:
train= df_train[['ltv','disbursed_amount','PERFORM_CNS.SCORE.DESCRIPTION','PERFORM_CNS.SCORE','State_ID','VoterID_flag',
              'NO.OF_INQUIRIES','Aadhar_flag','PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','PRI.NO.OF.ACCTS','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
              'NEW.ACCTS.IN.LAST.SIX.MONTHS','Employment.Type','PRI.CURRENT.BALANCE','Employee_code_ID','asset_cost','PRI.SANCTIONED.AMOUNT',
              'PRI.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','loan_default']]
test= df_test[['ltv','disbursed_amount','PERFORM_CNS.SCORE.DESCRIPTION','PERFORM_CNS.SCORE','State_ID','VoterID_flag',
              'NO.OF_INQUIRIES','Aadhar_flag','PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','PRI.NO.OF.ACCTS','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
              'NEW.ACCTS.IN.LAST.SIX.MONTHS','Employment.Type','PRI.CURRENT.BALANCE','Employee_code_ID','asset_cost','PRI.SANCTIONED.AMOUNT',
              'PRI.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT']]

In [21]:
train.head()

,ltv,disbursed_amount,PERFORM_CNS.SCORE.DESCRIPTION,PERFORM_CNS.SCORE,State_ID,VoterID_flag,NO.OF_INQUIRIES,Aadhar_flag,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,...,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NEW.ACCTS.IN.LAST.SIX.MONTHS,Employment.Type,PRI.CURRENT.BALANCE,Employee_code_ID,asset_cost,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,loan_default
0,89.55,50578,13,0,6,0,0,1,0,0,...,0,0,0,0,1588,58400,0,0,0,0
1,73.23,47145,8,598,6,0,0,1,1,1,...,1,0,1,27600,1588,65550,50200,50200,1991,1
2,89.63,53278,13,0,6,0,0,1,0,0,...,0,0,1,0,1588,61360,0,0,0,0
3,88.48,57513,11,305,6,0,1,1,0,0,...,0,0,1,0,1588,66113,0,0,31,1
4,88.39,52378,13,0,6,0,1,1,0,0,...,0,0,1,0,1588,60300,0,0,0,1


In [22]:
# apply scaler to the data
y=train['loan_default']
x=train.drop('loan_default',1)

In [23]:
#standard scaler on x and test
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(x)
# Apply transform to both the training set and the test set.
x_scale= pd.DataFrame(scaler.transform(x))
# x_over_scale= scaler.transform(x_over_smote)
test = scaler.transform(test)

/Users/gagicha/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/gagicha/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/Users/gagicha/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [24]:
train_x, test_x, train_y, test_y= model_selection.train_test_split(x_scale, y, test_size=0.33)

In [25]:
train_y.value_counts()

0    122292
1     33921
Name: loan_default, dtype: int64

In [26]:
#upsample
#downsample
train=pd.concat([train_x, train_y], axis=1)
count_0, count_1=train['loan_default'].value_counts()
train_0=train[train['loan_default']==0]
train_1=train[train['loan_default']==1]

df_over=train_1.sample(count_0, replace=True)
df_over=pd.concat([df_over,train_0], axis=0)
df_over['loan_default'].value_counts()

1    122292
0    122292
Name: loan_default, dtype: int64

In [27]:
train_y=df_over['loan_default']
train_x=df_over.drop('loan_default', axis=1)  

In [39]:
test_y.value_counts()

0    60251
1    16690
Name: loan_default, dtype: int64

In [40]:
xgb=XGBClassifier()
xgb.fit(train_x, train_y)
print('acc: ',xgb.score(test_x, test_y))
pred_xgb= xgb.predict(test_x)
print('auc: ',metrics.roc_auc_score(test_y, pred_xgb))
print(metrics.confusion_matrix(test_y, pred_xgb))

acc:  0.5750120221988276
auc:  0.6059427732113947
[[33217 27034]
 [ 5665 11025]]


In [35]:
rf=ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_x, train_y)
print('acc: ',rf.score(test_x, test_y))
pred_rf=rf.predict(test_x)
print('auc: ',metrics.roc_auc_score(test_y, pred_rf))
print(metrics.confusion_matrix(test_y, pred_rf))

acc:  0.7580483747286882
auc:  0.5284403768302243
[[56274  3977]
 [14639  2051]]


In [44]:
from sklearn.ensemble import ExtraTreesClassifier
ExtraTreesClassifier = ExtraTreesClassifier(n_estimators=1000)
ExtraTreesClassifier.fit(train_x, train_y)
y_pred = ExtraTreesClassifier.predict(test_x)
print('acc: ',ExtraTreesClassifier.score(test_x, test_y))
print('auc: ', metrics.roc_auc_score(test_y, y_pred))
print(metrics.confusion_matrix(test_y, y_pred))

acc:  0.768263994489284
auc:  0.5163359706561682
[[57920  2331]
 [15499  1191]]


In [48]:
out=pd.concat([df_test['UniqueID'], pd.DataFrame(rf1.predict(pd.DataFrame(test)),columns=['loan_default'])], axis=1)

In [49]:
out.head()

,UniqueID,loan_default
0,655269,0
1,723482,0
2,758529,0
3,763449,0
4,708663,0


In [38]:
out.to_csv('loan_predict.csv', encoding='utf-8', index=False)